In [34]:
# Importing libraries
import pandas as pd
import numpy as np
from dash import dash, dcc, html, Input, Output
import plotly.express as px


In [35]:
# Getting the data
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [36]:
# Getting column names and payload range
print(spacex_df.columns)
max_payload = spacex_df["Payload Mass (kg)"].max()
min_payload = spacex_df["Payload Mass (kg)"].min()



Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')


In [37]:
# Creating the app skeleton
app = dash.Dash(__name__)
server = app.server

uniquelaunchsites = spacex_df["Launch Site"].unique().tolist()
lsites = []
lsites.append({"label": "All Sites", "value": "All Sites"})
for site in uniquelaunchsites:
    lsites.append({"label": site, "value": site})


In [38]:
# Adding the layout
app.layout = html.Div(
    children=[
        html.H1(
            "SpaceX Launch Records Dashboard",
            style={"textAlign": "center", "color": "#503D36", "font-size": 40},
        ),
        dcc.Dropdown(
            id="site_dropdown",
            options=lsites,
            placeholder="Select a LaunchSite here",
            searchable=True,
            value="All Sites",
        ),
        html.Br(),
        html.Div(dcc.Graph(id="success-pie-chart")),
        html.Br(),
        html.P("Payload range (Kg):"),
        dcc.RangeSlider(
            id="payload_slider",
            min=0,
            max=10000,
            step=1000,
            marks={
                0: "0 kg",
                1000: "1000 kg",
                2000: "2000 kg",
                3000: "3000 kg",
                4000: "4000 kg",
                5000: "5000 kg",
                6000: "6000 kg",
                7000: "7000 kg",
                8000: "8000 kg",
                9000: "9000 kg",
                10000: "10000 kg",
            },
            value=[min_payload, max_payload],
        ),
        html.Div(dcc.Graph(id="success-payload-scatter-chart")),
    ]
)


In [40]:
# Adding Plots
@app.callback(
    Output(component_id="success-pie-chart", component_property="figure"),
    Input(component_id="site_dropdown", component_property="value")
)
def update_graph(site_dropdown):
    if site_dropdown == "All Sites":
        df = spacex_df[spacex_df["class"] == 1]
        fig = px.pie(
            df,
            names="Launch Site",
            hole=0.3,
            title="Total Success Launches By all sites",
        )
    else:
        df = spacex_df.loc[spacex_df["Launch Site"] == site_dropdown]
        fig = px.pie(
            df,
            names="class",
            hole=0.3,
            title="Total Success Launches for site " + site_dropdown,
        )
    return fig


In [39]:
# Adding plots
@app.callback(
    Output(component_id="success-payload-scatter-chart", component_property="figure"),
    [
        Input(component_id="site_dropdown", component_property="value"),
        Input(component_id="payload_slider", component_property="value"),
    ],
)
def update_scattergraph(site_dropdown, payload_slider):
    if site_dropdown == "All Sites":
        low, high = payload_slider
        df = spacex_df
        mask = (df["Payload Mass (kg)"] > low) & (df["Payload Mass (kg)"] < high)
        fig = px.scatter(
            df[mask],
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version",
            size="Payload Mass (kg)",
            hover_data=["Payload Mass (kg)"],
        )
    else:
        low, high = payload_slider
        df = spacex_df.loc[spacex_df["Launch Site"] == site_dropdown]
        mask = (df["Payload Mass (kg)"] > low) & (df["Payload Mass (kg)"] < high)
        fig = px.scatter(
            df[mask],
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version",
            size="Payload Mass (kg)",
            hover_data=["Payload Mass (kg)"],
        )
    return fig

In [29]:
if __name__ == "__main__":
    app.run_server(debug=False)

In [33]:
spacex_df.to_csv(r"C:\Users\sahaj\Downloads\spacex_launch_dashboard.csv")